# Database Admin 101 - Lab

## Introduction 

In this lab you'll go through the process of designing and creating a database. From there, you'll also begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Create a SQL database
* Create a SQL table
* Create rows in a SQL table
* Alter entries in a SQL table
* Delete entries in a SQL table
* Commit changes via sqlite3


## The Scenario

You are looking to design a database for a school which will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database and record some of your ideas below.

# Your Answer Here

Record some information here about how you would design such a database. What table would you include? What columns would each table have? What would be the primary means to join said tables?

- Student name
- Grades
- Contact information
- Year

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
#Your code here; import necessary packages
import sqlite3

In [2]:
#Your code here; create the database school.sqlite
conn = sqlite3.connect('school.sqlite')
c = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), email, telephone number, street, city, state and zipcode. Be sure to also create a primary key for the table. 

In [4]:
c.execute('''CREATE TABLE contactInfo ( 
                                    id INTEGER PRIMARY KEY,
                                    firstName TEXT,
                                    lastName TEXT,
                                    role TEXT,
                                    email TEXT,
                                    telephone INTEGER,
                                    street TEXT,
                                    city TEXT,
                                    state TEXT,
                                    zipcode INTEGER
                                    )
            ''')

In [5]:
# c.execute('''DROP TABLE contactInfo''')

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [6]:
# Code to load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [7]:
#Your code here
print("length: ",len(contacts))
contacts[0]

length:  8


{'firstName': 'Christine',
 'lastName': 'Holden',
 'role': 'staff',
 'telephone ': 2035687697,
 'street': '1672 Whitman Court',
 'city': 'Stamford',
 'state': 'CT',
 'zipcode ': '06995'}

In [8]:
# Your code to iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    firstName = contact['firstName']
    lastName = contact['lastName']
    role = contact['role']
    telephone = contact['telephone ']
    street = contact['street']
    city = contact['city']
    state = contact['state']
    zipcode = contact['zipcode ']
    c.execute("""INSERT INTO contactInfo (firstName, lastName, role, telephone, street, city, state, zipcode)
                 VALUES ('{}','{}','{}','{}','{}','{}','{}','{}');
                 """.format(firstName, lastName, role, telephone, street, city, state, zipcode))
                 

In [9]:
# Query the Table to Ensure it is populated
# Your code here
import pandas as pd
c.execute('''SELECT * FROM contactInfo;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,id,firstName,lastName,role,email,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,None,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,None,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,None,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,None,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,None,3259909290,1461 Briarhill Lane,Abilene,TX,79602


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [10]:
#Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [12]:
#Your code here; create the grades table.
c.execute("""CREATE TABLE grades (userId INTEGER NOT NULL,
                                  courseId INTEGER NOT NULL,
                                  grade INTEGER,
                                  PRIMARY KEY(userId, courseId)
                                  );
            """)

In [13]:
# c.execute("""DROP TABLE grades""")

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [14]:
#Your code here; find the duplicate entry
c.execute("""SELECT firstName, lastName, telephone, count(*) 
             FROM contactInfo
             GROUP BY 1,2,3
             HAVING count(*) > 1""")
c.fetchall()

[('Jane', 'Evans', 3259909290, 2)]

In [15]:
#Your code here; find the duplicate entry
c.execute("""SELECT id, firstName, lastName, telephone 
             FROM contactInfo
             WHERE telephone=3259909290;""")
c.fetchall()

[(5, 'Jane', 'Evans', 3259909290), (6, 'Jane', 'Evans', 3259909290)]

In [16]:
#Your code here; delete the duplicate entry
c.execute("""DELETE FROM contactInfo
             WHERE id = 6;""")

In [17]:
#Your code here; check that the duplicate entry was removed.
c.execute("""SELECT firstName, lastName, telephone, count(*) 
             FROM contactInfo
             GROUP BY 1,2,3
             HAVING count(*) > 1""")
c.fetchall()

[]

## Updating an Address

Ed Lyman just moved to 2910 Simpson Avenue York, PA 17403. Update his address accordingly.

In [18]:
#Your code here; update Ed's address
c.execute("""UPDATE contactInfo
             SET street = '2910 Simpson Avenue',
                 city = 'York',
                 state = 'PA',
                 zipcode = 17403
             WHERE firstName = 'Ed' and lastName ='Lyman';
                """)

In [19]:
c.execute("""SELECT * FROM contactInfo""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,id,firstName,lastName,role,email,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,None,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,None,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,None,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,None,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,None,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,None,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,None,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [20]:
#Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you certainly got a taste for creating, populating and maintaining databases! Feel free to continue fleshing out this exercise to continue practicing. 